In [ ]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [80]:
# Libraries to use
from __future__ import division 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.cluster import KMeans
import talib as tb
from scipy import stats

In [2]:
%matplotlib inline
%load_ext line_profiler

In [6]:
path = 'C:/Users/LZJF_02/Desktop/BacktestRA/'

In [71]:
filenameKKTest = 'KkRatioStrategy20170601-20171201.csv'
filenameKKTrain = 'KkRatioStrategy20160601-20170601.csv'

In [77]:
kkTest = pd.read_csv(path+filenameKKTest,infer_datetime_format=True,parse_dates=[0])
kkTrain = pd.read_csv(path+filenameKKTrain,infer_datetime_format=True,parse_dates=[0])
kkTest = calPosPeriod(kkTest)
kkTrain = calPosPeriod(kkTrain)

In [73]:
def calPosPeriod(df,barBin = 5):
    df.entryDt = df.entryDt.apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    df.exitDt = df.exitDt.apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    df['posPeriod'] = (df.exitDt - df.entryDt)
    df['posPeriod'] = df['posPeriod'].apply(lambda x: x.total_seconds() / 60 /barBin)
    return df

In [78]:
kkTest.describe()

,exitPrice,volume,turnover,commission,slippage,pnl,posPeriod
count,96.000000,96.000000,96.000000,96.000000,96.0,96.000000,96.000000
mean,3706.927083,0.145833,74071.041667,22.221312,20.0,23.612021,93.414583
std,228.328583,0.994502,4509.507495,1.352852,0.0,430.692289,136.597620
min,2959.000000,-1.000000,59440.000000,17.832000,20.0,-413.877000,1.600000
25%,3615.500000,-1.000000,72347.500000,21.704250,20.0,-275.162750,11.350000
50%,3727.000000,1.000000,74505.000000,22.351500,20.0,-92.665000,61.000000
75%,3845.250000,1.000000,76892.500000,23.067750,20.0,167.788000,104.500000
max,4098.000000,1.000000,81670.000000,24.501000,20.0,2426.381000,860.000000


In [79]:
kkTrain.describe()

,exitPrice,volume,turnover,commission,slippage,pnl,posPeriod
count,197.000000,197.000000,197.000000,197.000000,197.0,197.000000,197.000000
mean,2858.796954,0.197970,57161.776650,17.148533,20.0,52.242330,122.892386
std,421.791877,0.982706,8436.807622,2.531042,0.0,379.657858,276.142184
min,2055.000000,-1.000000,40920.000000,12.276000,20.0,-387.625000,1.000000
25%,2467.000000,-1.000000,49400.000000,14.820000,20.0,-208.063000,7.200000
50%,2968.000000,1.000000,59500.000000,17.850000,20.0,-65.231000,43.200000
75%,3170.000000,1.000000,63550.000000,19.065000,20.0,212.285000,129.000000
max,3655.000000,1.000000,73410.000000,22.023000,20.0,2630.305000,2876.000000


In [46]:
(datetime.strptime(kkTrain.exitDt.iloc[1], '%Y-%m-%d %H:%M:%S') - datetime.strptime(kkTrain.entryDt.iloc[1], '%Y-%m-%d %H:%M:%S')).total_seconds()

84240.0

In [25]:
kkTrain

,entryPrice,exitPrice,entryDt,exitDt,volume,turnover,commission,slippage,pnl
0,2155.0,2167.0,2016-06-13 10:55:00,2016-06-13 14:55:00,1.0,43220.0,12.966,20.0,87.034
1,2146.0,2083.0,2016-06-14 09:40:00,2016-06-15 09:04:00,-1.0,42290.0,12.687,20.0,597.313
2,2093.0,2077.0,2016-06-15 09:35:00,2016-06-15 10:55:00,1.0,41700.0,12.510,20.0,-192.510
3,2072.0,2074.0,2016-06-15 11:15:00,2016-06-15 14:15:00,-1.0,41460.0,12.438,20.0,-52.438
4,2087.0,2074.0,2016-06-15 21:05:00,2016-06-15 21:36:00,1.0,41610.0,12.483,20.0,-162.483
5,2037.0,2055.0,2016-06-16 22:15:00,2016-06-17 09:01:00,-1.0,40920.0,12.276,20.0,-212.276
6,2075.0,2078.0,2016-06-17 21:15:00,2016-06-20 09:00:00,1.0,41530.0,12.459,20.0,-2.459
7,2093.0,2094.0,2016-06-21 09:15:00,2016-06-21 11:00:00,1.0,41870.0,12.561,20.0,-22.561
8,2080.0,2096.0,2016-06-21 13:40:00,2016-06-21 21:10:00,-1.0,41760.0,12.528,20.0,-192.528
9,2118.0,2119.0,2016-06-22 09:10:00,2016-06-22 11:27:00,1.0,42370.0,12.711,20.0,-22.711


In [13]:
kkTest.describe()

,exitPrice,volume,turnover,commission,slippage,pnl
count,96.000000,96.000000,96.000000,96.000000,96.0,96.000000
mean,3706.927083,0.145833,74071.041667,22.221312,20.0,23.612021
std,228.328583,0.994502,4509.507495,1.352852,0.0,430.692289
min,2959.000000,-1.000000,59440.000000,17.832000,20.0,-413.877000
25%,3615.500000,-1.000000,72347.500000,21.704250,20.0,-275.162750
50%,3727.000000,1.000000,74505.000000,22.351500,20.0,-92.665000
75%,3845.250000,1.000000,76892.500000,23.067750,20.0,167.788000
max,4098.000000,1.000000,81670.000000,24.501000,20.0,2426.381000


In [14]:
kkTrain .describe()

,exitPrice,volume,turnover,commission,slippage,pnl
count,197.000000,197.000000,197.000000,197.000000,197.0,197.000000
mean,2858.796954,0.197970,57161.776650,17.148533,20.0,52.242330
std,421.791877,0.982706,8436.807622,2.531042,0.0,379.657858
min,2055.000000,-1.000000,40920.000000,12.276000,20.0,-387.625000
25%,2467.000000,-1.000000,49400.000000,14.820000,20.0,-208.063000
50%,2968.000000,1.000000,59500.000000,17.850000,20.0,-65.231000
75%,3170.000000,1.000000,63550.000000,19.065000,20.0,212.285000
max,3655.000000,1.000000,73410.000000,22.023000,20.0,2630.305000


In [115]:
pValueList = []
for i in xrange(10000):
    pValueList.append(stats.ttest_rel(np.random.choice(kkTrain.pnl.values,min(kkTest.shape[0],kkTrain.shape[0])), kkTest.pnl.values).pvalue)   
pValueSeries = pd.Series(pValueList)
pValueSeries.describe()

count    10000.000000
mean         0.570841
std          0.260259
min          0.003174
25%          0.360302
50%          0.586430
75%          0.792447
max          0.999971
dtype: float64

In [116]:
pValueSeries.quantile(0.05)

0.13788764978952164

In [117]:
# 根据pValue的均值和中位数以及5%的分位数可以看出 P-Value是大概率远大于0.05的，所以接受原假设，及两组PNL方差相同，收益稳定。